In [41]:
from statsbombpy import sb #Manage stats bomb data
import pandas as pd #Work with any type of data
from mplsoccer import Pitch,Sbopen #Visualize soccer data
import matplotlib.pyplot as plt #Math graphs and stuff like that. Not sure if needed
import numpy as np #Work with matrices and number precisly

MexVSArgID= 3857289
MexVsArabID = 3857260
worldCupID = 43
seasonID = 106

parser = Sbopen()
df,related,freezer,tactics = parser.event(MexVSArgID) #Get info about the match (temach xdddd)

df = df[df['team_name']=='Mexico'] #Data from Mexico only
passes = df[df['type_name']=='Pass'] #Passes from Mexico
filteredPasses = passes[['player_name','x','y','end_x', 'end_y','pass_recipient_name','outcome_id','outcome_name']] #Data I need from the passes
successfullPasses = filteredPasses[filteredPasses['outcome_name'].isnull()] #Only successful passes

#Herramienta sorpresa que nos ayudará más tarde
playersName = successfullPasses['player_name'].unique().tolist() 
playersID = {}

for index in range(len(playersName)):
    playersID[playersName[index]] = index

successfullPasses = successfullPasses.reset_index(drop=True) #New 0 to n id
# for debugging print(successfullPasses.shape)

print(playersID)

{'José Andrés Guardado Hernández': 0, 'Jesús Daniel Gallardo Vasconcelos': 1, 'Héctor Alfredo Moreno Herrera': 2, 'Héctor Miguel Herrera López': 3, 'Néstor Alejandro Araújo Razo': 4, 'Kevin Nahin Álvarez Campos': 5, 'Ernesto Alexis Vega Rojas': 6, 'Luis Gerardo Chávez Magallón': 7, 'Francisco Guillermo Ochoa Magaña': 8, 'César Jasib Montes Castro': 9, 'Hirving Rodrigo Lozano Bahena': 10, 'Érick Gabriel Gutiérrez Galaviz': 11, 'Raúl Alonso Jiménez Rodríguez': 12, 'Carlos Uriel Antuna Romero': 13, 'Roberto Carlos Alvarado Hernández': 14}


In [44]:
#Prepares the adjc matrix
adjcMatrix = []
playersMexico = 15

for _ in range(playersMexico): #You use _ when you don't care about the index value
    adjcMatrix.append([])

for i in range(playersMexico): #i goes from 0 to 14 (playersMexico - 1)
    for _ in range(playersMexico):
        adjcMatrix[i].append(0)

#Iterates over each row in successfulPasses
for index, row in successfullPasses.iterrows():
    passer = row['player_name']  
    receiver = row['pass_recipient_name']
    adjcMatrix[playersID[passer]][playersID[receiver]] += 1
    start_pos = (row['x'], row['y'])  
    end_pos = (row['end_x'], row['end_y']) 
    print(f"Pase de {index} {passer} a {receiver} desde {start_pos} hasta {end_pos} xd")

Pase de 0 José Andrés Guardado Hernández a Héctor Miguel Herrera López desde (61.0, 40.1) hasta (43.3, 36.5) xd
Pase de 1 Jesús Daniel Gallardo Vasconcelos a Héctor Alfredo Moreno Herrera desde (38.4, 0.1) hasta (26.5, 9.7) xd
Pase de 2 Héctor Alfredo Moreno Herrera a Hirving Rodrigo Lozano Bahena desde (26.5, 9.4) hasta (63.4, 18.1) xd
Pase de 3 Héctor Miguel Herrera López a Néstor Alejandro Araújo Razo desde (96.2, 29.8) hasta (81.6, 59.3) xd
Pase de 4 Néstor Alejandro Araújo Razo a Kevin Nahin Álvarez Campos desde (83.5, 64.0) hasta (99.5, 74.0) xd
Pase de 5 Kevin Nahin Álvarez Campos a Néstor Alejandro Araújo Razo desde (98.7, 73.2) hasta (84.0, 68.0) xd
Pase de 6 Néstor Alejandro Araújo Razo a Kevin Nahin Álvarez Campos desde (84.0, 68.0) hasta (97.6, 74.5) xd
Pase de 7 Kevin Nahin Álvarez Campos a Néstor Alejandro Araújo Razo desde (91.6, 76.7) hasta (75.6, 71.8) xd
Pase de 8 Néstor Alejandro Araújo Razo a Kevin Nahin Álvarez Campos desde (73.5, 71.3) hasta (86.8, 75.1) xd
Pase d

In [47]:
for player in adjcMatrix:
    print(player)

sum = 0
for passer in adjcMatrix:
    for passed in passer:
        sum += passed
print(sum)
#Btw, Héctor Herrera is in index 2

[0, 2, 4, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0]
[3, 0, 6, 1, 0, 0, 3, 1, 1, 2, 0, 1, 0, 0, 1]
[2, 6, 0, 3, 0, 1, 0, 0, 5, 7, 2, 8, 2, 0, 2]
[2, 1, 1, 0, 10, 1, 0, 6, 2, 7, 1, 3, 1, 2, 0]
[0, 0, 1, 5, 0, 6, 1, 5, 1, 19, 2, 0, 5, 2, 0]
[0, 0, 0, 0, 6, 0, 2, 5, 0, 0, 2, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0]
[1, 0, 1, 2, 9, 3, 0, 0, 0, 1, 1, 1, 0, 0, 2]
[2, 1, 6, 1, 0, 0, 0, 2, 0, 8, 0, 0, 0, 0, 0]
[0, 1, 11, 6, 11, 0, 0, 0, 8, 0, 0, 8, 1, 2, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0]
[0, 3, 5, 6, 1, 0, 0, 0, 0, 4, 1, 0, 0, 0, 5]
[0, 1, 0, 2, 0, 0, 0, 2, 0, 0, 1, 0, 0, 2, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0]
304
